In [ ]:
import sys
from pathlib import Path

# Flexible imports to allow for development without installation 
#  ** Use this version with Jupyter Notebooks **
try:
    # Try to import from the installed package
    from ras_commander import init_ras_project, RasHdf, RasExamples, RasCmdr, RasPlan, RasGeo, RasUnsteady, RasUtils, RasPrj, ras
except ImportError:
    # If the import fails, add the parent directory to the Python path
    import os
    current_file = Path(os.getcwd()).resolve()
    parent_directory = current_file.parent
    sys.path.append(str(parent_directory))
    
    # Now try to import again
    from ras_commander import init_ras_project, RasHdf, RasExamples, RasCmdr, RasPlan, RasGeo, RasUnsteady, RasUtils, RasPrj, ras

print("ras_commander imported successfully")


import pandas as pd
from pathlib import Path
import psutil
from ras_commander import RasExamples, RasCmdr, init_ras_project, RasPrj, RasUtils, ras

# Load the list of projects from the CSV file
projects_df = pd.read_csv("example_projects.csv")

# Initialize the RasExamples instance
ras_examples = RasExamples()

# Function to process a single project
def process_project(project_name):
    # Extract the project using RasExamples
    extracted_paths = ras_examples.extract_project([project_name])
    if not extracted_paths:
        raise ValueError(f"Failed to extract project: {project_name}")
    extracted_path = extracted_paths[0]
    # Initialize the project
    try:
        project = RasPrj()
        project = init_ras_project(ras_project_folder=Path(extracted_path), ras_version="6.5")
    except Exception as e:
        print(f"Failed to initialize project {project_name}: {str(e)}")
        return

    # Determine optimal core usage
    physical_cores = psutil.cpu_count(logical=False)
    print(f"Physical cores: {physical_cores}")
    print(f"Adjust the number of cores per plan and max workers as needed.")

    # Run all plans in parallel
    try:
        results = RasCmdr.compute_parallel(
            max_workers=2,
            num_cores=2,
            ras_object=project,
        )
    except Exception as e:
        print(f"Failed to run plans for project {project_name}: {str(e)}")
        return

    # Log the results
    for plan, success in results.items():
        print(f"Project: {project_name}, Plan: {plan}, Success: {success}")

    # Clean up extracted project
    RasUtils.remove_with_retry(Path(extracted_path), is_folder=True)

# Process each project listed in the CSV
for _, row in projects_df.iterrows():
    project_name = row['Project']

    process_project(project_name)
    print(f"Processed project: {project_name}")


In [ ]:
import pandas as pd
from pathlib import Path
import psutil
from ras_commander import RasCmdr, init_ras_project, RasPrj, RasUtils, ras

print("Starting script execution")

# Function to process a single project
def process_project(project_path):
    project_name = project_path.name
    print(f"Processing project: {project_name}")
    # Initialize the project
    try:
        print(f"Initializing project: {project_name}")
        project = RasPrj()
        project = init_ras_project(ras_project_folder=project_path, ras_version="6.5")
        print(f"Project {project_name} initialized successfully")
    except Exception as e:
        print(f"Failed to initialize project {project_name}: {str(e)}")
        return

    # Determine optimal core usage
    physical_cores = psutil.cpu_count(logical=False)
    print(f"Physical cores: {physical_cores}")
    print(f"Adjust the number of cores per plan and max workers as needed.")

    # Run all plans in parallel
    try:
        print(f"Starting parallel computation for project: {project_name}")
        results = RasCmdr.compute_parallel(
            max_workers=2,
            num_cores=2,
            ras_object=project,
        )
        print(f"Parallel computation completed for project: {project_name}")
    except Exception as e:
        print(f"Failed to run plans for project {project_name}: {str(e)}")
        return

    # Log the results
    print(f"Logging results for project: {project_name}")
    for plan, success in results.items():
        print(f"Project: {project_name}, Plan: {plan}, Success: {success}")

# Get user input for the parent folder
parent_folder = Path(r"C:\GH\ras-commander\examples\example_projects")
print(f"Parent folder set to: {parent_folder}")

# Process each second-level subfolder as a project
print("Starting to process projects")
for category_folder in parent_folder.iterdir():
    if category_folder.is_dir():
        print(f"Processing category: {category_folder.name}")
        for project_folder in category_folder.iterdir():
            if project_folder.is_dir():
                print(f"Found project folder: {project_folder}")
                process_project(project_folder)
                print(f"Processed project: {project_folder.name}")

print("Script execution completed")
